In [1]:
import sys
import string
sys.path.append("..")

# Importamos las funciones necesarias para correr los modelos
from icg.processing import (text, image, dataset)
from icg.model import model

from keras import backend as K
from keras.models import load_model
K.clear_session()

Using TensorFlow backend.


<h1 style="width: 100%;text-align:center;"> Generación de subtítulos  de imagen usando aprendizaje profundo.  </h1>
<p style="width: 100%;text-align:center;">  Universidad de Antioquia <br> Angelower Santana Velasquez <br> Martin Elias Quintero  Osorio</p>

<p>Las redes neuronales convolucionales representan actualmente la tecnologia por defecto en el reconocimiento de imagenes. Al realizar un paralelo para las estrategias usadas en el aprendizaje profundo para el reconocimiento de secuencias apareceria el nombre de las redes neuronales recurrentes. Estas ultimas, logran como resultado generalizar informacion temporal (secuencial) que se encuentra representada en los datos. Entre la forma de los datos secuenciales se encuentra: Series de tiempo, texto, voz, etc..</p>

<p>Si bien, por un lado aprovechamos el potencial de las redes ocultas convolucionales para obtenerar una representacion de las caracteristicas de las imagenes, por otro, la informacion temporal es almacenada en cada unidad neuronal de las redes recurrentes. Cada una por su lado han demostrado la capacidad. El presente trabajo busca explorar una tecnica que combina ambas arquitecturas y realizar distintos experimentos donde se intentara mejorar el rendimiento de la arquitectura final.</p>

<h1 style="width: 100%;text-align:center;"> El problema de la generacion de subtitulos </h1>



<p>Consideremos la siguiente imagen.</p>

<img style="float: right;" src="https://www.gopetplan.com/sites/default/files/2018-08/rsz_shutterstock_261121304.jpg" alt="Drawing" style="width: 100px;"> <p style="width: 100%;text-align:center;"><b><i>imagen 1 </i></b> 

<p>Es probable que al rntentar realizar predicciones de la <b><i>Imagen 1</i></b> en una red convolucional la respuesta 
sea <b>perro</b>. Otras opciones de posible respuesta puede ser <b>cachorro</b> o con un nivel de probabilidad mucho menor <b>helado</b>. Este es un problema clásico de clasificación. Internamente la CNN es capaz de producir una representación de la imagen(extracción de características) y desde allí identificar la clase a la que pertenece dando valores de probabilidad a una cantidad definida de neuronas de salida(normalmente una capa softmax).</p>

<p> Otra respuesta que es interesante seria lograr que la salida de la red logre decirnos las acciones que ocurren en la imagen, es decir, en vez de decirnos <b>perro</b> ,<b>cachorro</b> o <b>helado</b> la respuesta sera : <b>Cachorros comiendo helado</b>, <b>Dos cachorros lamiendo un helado de vainilla</b> o <b>Dos cachorros sobre una mesa de colores</b>.A este tipo de problemas se le conoce como <b>Image Caption Generation</b> o <b>Generación de subtítulos</b>           </p>

<p> La idea principal es utilizar la potencia del reconocimiento de las CNN con el desempeño de las RNN para generar texto.</p>

<h1 style="width: 100%;text-align:center;"> Arquitecturas : Fusión e inyección </h1>



<h1 style="width: 100%;text-align:center;">Fusión </h1>



<img style="float: right;" src="./img/fusion.png" alt="Drawing" style="width: 100px;"> <p style="width: 100%;text-align:center;"> 

Para obtener una representación lingüística de la frase de entrada realizamos un procesamiento a priori mediante técnicas de procesamiento de lenguaje natural (Tokenizar, embeddings...), luego, usamos una red LSTM con n neuronas para lograr que el sistema adquiera la habilidad de reconocer secuencias. Paralelamente, para la imagen, modificamos las capas finales de una red convolución entrenada con 'imagenet', esta modificación es eliminar la capa softmax del final que es utilizada para detectar la clase con mayor probabilidad. Para las versiones 19 y 16 de VGG esto quiere decir que al final (luego de hacer el aplanamiento) obtenemos un vector con longitud 4096. Nuestro objetivo es combinar ambas representaciones, lingüística y visual, sumando los vectores  de las representaciones o uniendo uno tras el otro. Por ello, la red densa de la capa convolucional debe corresponder en tamaño (numero de neuronas) con la cantidad de neuronas de la LSTM. Finalmente pasamos la representación combinada por una capa softmax que aprenderá a discriminar la palabra mas probable. Las secuencias en el preprocesamiento cuentan con dos gatillos: Uno para iniciar una secuencia(< seqinit >) y otro para saber cuando se finaliza(< seqend >). La predicciónn de las secuencias es una a una. 

<h1 style="width: 100%;text-align:center;"> Inyección </h1>

Taken from: <a href="https://machinelearningmastery.com/caption-generation-inject-merge-architectures-encoder-decoder-model/">Machine Learning Mastery</a>

El modelo de inyección combina la forma codificada de la imagen con cada palabra de la descripción de texto generada hasta el momento.

Este modelo de Inyección utiliza una red neuronal recurrente como modelo de generación de texto, que usa una secuencia de información de imagen y palabra como entrada para generar la siguiente palabra en la secuencia.

En estas arquitecturas de "inyección", el vector de imagen (generalmente extraido de los valores de activación de una capa oculta en una red neuronal convolucional, en este caso la VGG) se inyecta en el RNN, por ejemplo, tratando el vector de imagen a la par con una "palabra" e incluyendo como parte del prefijo de subtítulo.

<img style="float: center;" src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Inject-Architecture-for-Encoder-Decoder-Model.png" alt="Drawing" style="width: 80%;"> 
<p style="width: 100%;text-align:center;"><b><i>imagen 2 </i></b> 


<h1 style="width: 100%;text-align:center;"> Preprocesamiento</h1>

Un buen conjunto de datos para usar al comenzar con el subtitulado de imágenes es el conjunto de datos Flickr8K.

El motivo es que es realista y relativamente pequeño para que pueda descargarlo y construir modelos en su estación de trabajo utilizando una CPU.


<h1 style="width: 100%;text-align:center;"> Texto</h1>

Se debe hacer un pre-tratamiento del texto que contiene las descripciones de las imágenes. Se cuenta con un dataset con 8092 imágenes y un conjunto de 40460 descripciones de las imágenes, cada imagen tiene 5 frases que la describen (ver imagen 3). Cada frace inicia con el código de identificación de la imagen, el cual es el nombre del archivo de la imagen, esta información está contenida en un archivo txt, por lo que no hay que procesar caracteres especiales. 

<img style="float: center;" src="./img/frases.png" alt="Drawing" style="width: 80%;"> 
<p style="width: 100%;text-align:center;"><b><i>imagen 3 </i></b> 

<ul> 
<li>La traducción del alemán original utiliza el inglés del Reino Unido (por ejemplo, " viajando ").</li>
<li>Las líneas están envueltas artificialmente con nuevas líneas de aproximadamente 70 caracteres (meh).</li>
<li>No hay errores tipográficos obvios o errores ortográficos.</li>
<li>Hay puntuaciones como comas, apóstrofes, comillas, signos de interrogación y más.</li>
<li>Hay descripciones con guiones como "como una armadura".</li>
<li>Se usan guiones ("-") para continuar las oraciones.</li>
<li>Hay nombres de personas.</li>
<li>No parece haber números que requieran manejo, como es el caso de años (2019)</li>
    <li>Hay marcadores de sección (por ejemplo, "II" y "III").</li> 
    <li>Se ha eliminado el primer "I".</li>
    </ul>
    
Los pasos a seguir para el pretratamiento del texto, se describen a continuación:
<ol>
    <li>Carga de datos</li>
    <li>Hacer split por espacios en blanco.</li>
    <li>Seleccionar palabras.</li>
    <li>Eliminar puntuación.</li>
    <li>Se vuelven minúsculas todas las letras.</li>
</ol>

Hay que destacar que aunque el pretratamiento del texto es una tarea muy pesada, para este caso en particular, el dataset ya está bien depurado, así que este proceso es reltivamente fácil.

<h1 style="width: 100%;text-align:center;"> Imagen </h1>

Se usan modelos pre-entrenado para extracción de características de las fotos. Hay muchos modelos para elegir. En este caso, utilizaremos los modelos de Oxford Visual Geometry Group, conocidas como VGG16 y VGG19. 

Como el interés de este trabajo no es la clasificación de las imágenes como lo hacer la VGG completa, se elimina la última capa del modelo y se extraen las características de la imagen.

<h1 style="width: 100%;text-align:center;"> Experimentos </h1>

In [2]:
import sys
import string
sys.path.append("..")

# Importamos las funciones necesarias para correr los modelos
from icg.processing import (text, image, dataset)
from icg.model import model

from keras import backend as K
from keras.models import load_model
K.clear_session()

In [3]:
TOKEN_TXT = "../src/dataset/Flickr8k_text/Flickr8k.token.txt"
IMAGES = '../src/dataset/Flickr8k_Dataset/Flicker8k_Dataset'
FEACTURES_PKL = "../src/features.pkl"

URL_TEXT = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip"
URL_IMAGES =  "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip"
TRAIN_TXT = '../src/dataset/Flickr8k_text/Flickr_8k.trainImages.txt'
TEXT_TXT = '../src/dataset/Flickr8k_text/Flickr_8k.testImages.txt'


In [4]:
dataset.get_dataset(URL_IMAGES, URL_TEXT, dest="../src/dataset")

It could take a few minutes
Unzip... ../src/dataset/Flickr8k_Dataset.zip in ../src/dataset/Flickr8k_Dataset 
It could take a few minutes
Unzip... ../src/dataset/Flickr8k_text.zip in ../src/dataset/Flickr8k_text 


In [5]:
# Preprocesando imagenes
# extract features from all images
K.clear_session()
features = image.extract_features(IMAGES)
print('Extracted Features: %d' % len(features))
# save to file
image.save_feactures(features,FEACTURES_PKL)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [6]:
# Preprocesando el texto 
# load descriptions
doc = text.load_doc(TOKEN_TXT)
# parse descriptions
descriptions = text.load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
text.clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = text.to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
text.save_descriptions(descriptions, 'descriptions.txt')

Loaded: 8092 
Vocabulary Size: 8763


In [7]:
train = text.load_set(TRAIN_TXT)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = text.load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = image.load_photo_features(FEACTURES_PKL, train)
print('Photos: train=%d' % len(train_features))

Dataset: 6000
Descriptions: train=6000
Photos: train=6000


### Experimentos mediante el modelo de  fusión


In [9]:
# prepare tokenizer
tokenizer = text.create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = text.max_length(train_descriptions)
print('Description Length: %d' % max_length)
# define the model
tmp_model = model.define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 5
steps = len(train_descriptions)
for i in range(epochs):
    # create the data generator
    generator = text.data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
    # fit for one epoch
    tmp_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    tmp_model.save('fmodel_' + str(i) + '.h5')



Vocabulary Size: 7579
Description Length: 34
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 34, 256)      1940224     input_5[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 4096)         0           input_4[0][0]                    
________________________________________________________________

In [11]:
# load test set
test = text.load_set(TEXT_TXT)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = text.load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = image.load_photo_features(FEACTURES_PKL, test)
print('Photos: test=%d' % len(test_features))

# load the model
filename = 'fmodel_4.h5'
my_model = load_model(filename)
# evaluate model
model.evaluate_model(my_model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.547386
BLEU-2: 0.274671
BLEU-3: 0.179011
BLEU-4: 0.075552


### Experimentos mediante el modelo de inyección



In [12]:
# prepare tokenizer
tokenizer = text.create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = text.max_length(train_descriptions)
print('Description Length: %d' % max_length)
# define the model
tmp_model = model.define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 5
steps = len(train_descriptions)
for i in range(epochs):
    # create the data generator
    generator = text.data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
    # fit for one epoch
    tmp_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    tmp_model.save('imodel_' + str(i) + '.h5')


Vocabulary Size: 7579
Description Length: 34
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 34, 256)      1940224     input_7[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 4096)         0           input_6[0][0]                    
________________________________________________________________

In [14]:
# load test set
test = text.load_set(TEXT_TXT)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = text.load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = image.load_photo_features(FEACTURES_PKL, test)
print('Photos: test=%d' % len(test_features))

# load the model
filename = 'imodel_4.h5'
my_model = load_model(filename)
# evaluate model
model.evaluate_model(my_model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.546388
BLEU-2: 0.279281
BLEU-3: 0.175566
BLEU-4: 0.070699
